<a href="https://colab.research.google.com/github/ikedachin/llm_lecture_2024/blob/main/inference_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install libraries

In [ ]:
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install --upgrade torch
!pip install --upgrade xformers

In [ ]:
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn


# import libraries

In [ ]:
import re
import json
from tqdm import tqdm
import torch
from unsloth import FastLanguageModel

# config

In [ ]:
max_seq_length = 1024
dtype = None
load_in_4bit = True
model_id = "ikedachin/llm-jp-3-13b-october-news-e1-all-3-5-sft-llmjp-magpie-20000"

In [ ]:
# set Token of Huggingface
from google.colab import userdata
HF_W=userdata.get('HF_TOKEN')

# download model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code = True,
)

# prepare a dataset for inference

In [ ]:
datasets = []
with open("./elyza-tasks-100-TV_0.jsonl", "r") as f:
    item = ""
    for line in f:
        line = line.strip()
        item += line
        if item.endswith("}"):
            datasets.append(json.loads(item))
            item = ""

# inference

In [ ]:
# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
    input = dt["input"]

    prompt = f"""### 指示
{input}
### 回答
"""

    inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True, do_sample=False, repetition_penalty=1.2)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

    results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

# save results

In [ ]:
json_file_id = re.sub(".*/", "", model_id)
with open(f"/content/{json_file_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')